# Random forests imputation

## Setup

### Imports

In [63]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
import os

## Data

Get PUF records.

In [64]:
df = pd.read_csv('puf2011.csv')

# Removes aggregate rows
df = df[(df.RECID != 999996) & 
        (df.RECID != 999997) & 
        (df.RECID != 999998) &
        (df.RECID != 999999)]

In [65]:
P22250_indp = ['DSI', 'EIC', 'MARS', 'E00200', 'E00300','E00600', 'E00800', 'E00900', 
               'E01100', 'E01400', 'E01500', 'E01700','E02100', 'E02300', 'E02400', 
               'E03150', 'E03210', 'E03240', 'E03270','E03300', 'E17500', 'E18400', 
               'E18500', 'E19200', 'E19800', 'E20100','E20400', 'E32800', 'P22250']
df.drop([i for i in df.columns if i not in P22250_indp], axis=1, inplace=True)

In [66]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
DSI,163786.0,0.037586,0.190192,0.0,0.0,0.0,0.0,1.0
EIC,163786.0,0.156588,0.540109,0.0,0.0,0.0,0.0,3.0
MARS,163786.0,1.880747,0.868159,1.0,1.0,2.0,2.0,4.0
E00200,163786.0,219531.584958,915171.923122,0.0,3350.0,30800.0,104800.0,56530000.0
E00300,163786.0,21342.225465,232007.404557,0.0,0.0,30.0,1290.0,29460000.0
E00600,163786.0,35211.744056,369615.898863,0.0,0.0,0.0,2050.0,37050000.0
E00800,163786.0,32.701025,1269.963802,0.0,0.0,0.0,0.0,174000.0
E00900,163786.0,16520.290611,236232.722974,-29990000.0,0.0,0.0,0.0,18990000.0
E01100,163786.0,27.428144,1348.768565,0.0,0.0,0.0,0.0,282700.0
E01400,163786.0,9755.219567,96146.012962,0.0,0.0,0.0,0.0,6424000.0


## Model

Train a random forests model.

In [67]:
YCOL = 'P22250'

In [68]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    df.drop(YCOL, axis=1), df[YCOL], random_state=3)

In [69]:
Y_train_sign = np.sign(Y_train)
Y_test_sign = np.sign(Y_test)

In [70]:
# Reduce for faster runtime.
N_ESTIMATORS = 100
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.
rf.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=3, verbose=True, warm_start=False)

### Model description

Show the ten most important features.

In [71]:
feature_importance = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importance.sort_values(ascending=False)[:10]

E00300    0.203959
E00200    0.129206
MARS      0.115906
E19200    0.080111
E00600    0.079555
E03300    0.057119
E18400    0.047471
E20400    0.042516
E19800    0.041598
E03270    0.038676
dtype: float64

## Predict

### Top-line (average)

In [72]:
pred = pd.DataFrame({'actual': Y_test,
                     'pred': rf.predict(X_test)})
pred['error'] = pred.pred - pred.actual
pred['actual_sign'] = np.sign(pred.actual)
pred['pred_sign'] = np.sign(pred.pred)
pred['correct_sign'] = (pred.actual_sign == pred.pred_sign)
pred['count'] = 1

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.8s finished


MAE, RMSE, and % negative/zero/positive.

In [73]:
pred.error.abs().mean()

34899.14858268668

In [74]:
pred.error.pow(2).mean() ** 0.5

318334.00580239546

In [75]:
pred.pivot_table(index='actual_sign', columns='pred_sign', values='count', 
                 aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,3829,66,1911,5806
0.0,9572,14211,6388,30171
1.0,3067,38,1865,4970
All,16468,14315,10164,40947


In [76]:
pred.correct_sign.mean()

0.48611619898893693

### All trees

In [77]:
preds = []
for estimator in rf.estimators_:
    preds.append(estimator.predict(X_test))
preds = np.array(preds).transpose()  # One row per record.

In [78]:
rand_col = np.random.randint(N_ESTIMATORS, size=preds.shape[0])
random_tree = preds[np.arange(preds.shape[0]), rand_col]

In [79]:
pred_random_tree = pd.DataFrame({'actual': Y_test,
                                 'pred': random_tree})
pred_random_tree['error'] = pred_random_tree.pred - pred_random_tree.actual
pred_random_tree['actual_sign'] = np.sign(pred_random_tree.actual)
pred_random_tree['pred_sign'] = np.sign(pred_random_tree.pred)
pred_random_tree['correct_sign'] = (
    pred_random_tree.actual_sign == pred_random_tree.pred_sign)
pred_random_tree['count'] = 1

As expected, MAE and RMSE exceed values from the point estimate prediction.

In [80]:
pred_random_tree.error.abs().mean()

47515.089149302665

In [81]:
pred_random_tree.error.pow(2).mean() ** 0.5

514707.9658813344

But the distribution of sign is closer to correct, since it's not averaging out the zeros.

In [82]:
pred_random_tree.pivot_table(index='actual_sign', columns='pred_sign', 
                             values='count', aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,2272,1753,1781,5806
0.0,2211,26060,1900,30171
1.0,1772,1625,1573,4970
All,6255,29438,5254,40947


In [83]:
pred_random_tree.correct_sign.mean()

0.7303343346276895

#### Log-loss of sign

In [84]:
preds_neg = np.sum(preds < 0, axis=1) / 100
preds_zero = np.sum(preds == 0, axis=1) / 100
preds_pos = np.sum(preds > 0, axis=1) / 100

rf_pred_proba = list(map(list, zip(*[preds_neg, preds_zero, preds_pos])))

metrics.log_loss(Y_test_sign, rf_pred_proba)

0.7124920018728914

## Multinomial logistic regression of sign

In [85]:
mult = linear_model.LogisticRegression(
    multi_class='multinomial', solver='newton-cg', random_state=3)
mult.fit(X_train, Y_train_sign)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=3, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

### Coefficients

p-values are tricky to calculate, so just show the features with highest absolute coefficient on the zero class.

In [86]:
mult_coef = pd.DataFrame({
    'negative': mult.coef_[0],
    'zero': mult.coef_[1],
    'positive': mult.coef_[2],
    'abs_zero': np.abs(mult.coef_[1])},
    index=X_train.columns)

mult_coef.sort_values('abs_zero', ascending=False)[:10]

,abs_zero,negative,positive,zero
MARS,0.053210,-0.026201,-0.027009,0.053210
E01100,0.021221,-0.010820,-0.010401,0.021221
EIC,0.008110,-0.004071,-0.004039,0.008110
DSI,0.002129,-0.001068,-0.001061,0.002129
E03210,0.001851,-0.000770,-0.001081,0.001851
E32800,0.000288,-0.000130,-0.000158,0.000288
E02300,0.000157,-0.000062,-0.000095,0.000157
E03150,0.000041,-0.000018,-0.000023,0.000041
E00800,0.000037,0.000011,-0.000049,0.000037
E02400,0.000028,-0.000009,-0.000019,0.000028


### Predict

In [87]:
mult_pred = pd.DataFrame({'actual': Y_test_sign,
                          'pred': mult.predict(X_test)})
mult_pred['sign_correct'] = (mult_pred.actual == mult_pred.pred)
mult_pred['count'] = 1
mult_pred.sign_correct.mean()

0.7609348670232251

In [88]:
mult_pred.pivot_table(index='actual', columns='pred', 
                      values='count', aggfunc=sum, margins=True)

pred,-1.0,0.0,1.0,All
actual,,,,
-1.0,2255,3372,179,5806
0.0,1342,28711,118,30171
1.0,1738,3040,192,4970
All,5335,35123,489,40947


#### Log-loss

In [89]:
mult_pred_proba = mult.predict_proba(X_test)
metrics.log_loss(Y_test_sign, mult_pred_proba)

0.920328648420079